In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [ ]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [ ]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [ ]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [ ]:
params = {
    "n_estimators": [100, 200, 300],
    "eta": [0.05, 0.1, 0.2],
    "gamma": [0, 1],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 0.9, 1.0],
    "colsample_bytree": [0.8, 0.9, 1.0],
    "booster": ["gbtree", "dart"],
    "scale_pos_weight": [1, 5]
}

xgb = XGBClassifier(random_state = 2025)

In [ ]:
np.random.seed(2025)
xgb_grid = GridSearchCV(xgb, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

Best Parameters: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.2, 'gamma': 0, 'max_depth': 3, 'n_estimators': 200, 'scale_pos_weight': 1, 'subsample': 0.8}


In [ ]:
np.random.seed(2025)
xgb_model = XGBClassifier(n_estimators = 200, eta = 0.2, gamma = 0, max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_model.fit(X_temp, y_temp)
y_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [ ]:
np.random.seed(2025)
xgb_model.fit(X_full, y)
y_pred_xgb = xgb_model.predict(X_test_full)

In [ ]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_xgb})

with open('XGB_Full_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [ ]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [ ]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [ ]:
np.random.seed(2025)
xgb_grid_rfe = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
xgb_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', xgb_grid_rfe.best_params_)

Best Parameters: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.2, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'scale_pos_weight': 1, 'subsample': 0.8}


In [ ]:
np.random.seed(2025)
xgb_model_rfe = XGBClassifier(n_estimators = 300, eta = 0.2, gamma = 1, max_depth = 5, subsample = 0.8, colsample_bytree = 0.8, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = xgb_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [ ]:
np.random.seed(2025)
xgb_model_rfe.fit(X_RFE, y)
y_pred_rfe_xgb = xgb_model_rfe.predict(X_RFE_test)

In [ ]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_xgb})

with open('XGB_RFE_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [ ]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [ ]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [ ]:
np.random.seed(2025)
xgb_grid_lasso = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
xgb_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', xgb_grid_lasso.best_params_)

Best Parameters: {'booster': 'dart', 'colsample_bytree': 1.0, 'eta': 0.1, 'gamma': 1, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 5, 'subsample': 0.9}


In [20]:
np.random.seed(2025)
xgb_model_lasso = XGBClassifier(n_estimators = 100, eta = 0.1, gamma = 1, max_depth = 3, subsample = 0.9, colsample_bytree = 1.0, booster = 'dart', scale_pos_weight = 5, random_state = 2025)
xgb_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = xgb_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [21]:
np.random.seed(2025)
xgb_model_lasso.fit(X_LASSO, y)
y_pred_lasso_xgb = xgb_model_lasso.predict(X_LASSO_test)

In [22]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_xgb})

with open('XGB_LASSO_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [23]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [24]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [25]:
np.random.seed(2025)
xgb_grid = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

Best Parameters: {'booster': 'gbtree', 'colsample_bytree': 0.9, 'eta': 0.2, 'gamma': 1, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 1.0}


In [26]:
np.random.seed(2025)
xgb_model = XGBClassifier(n_estimators = 100, eta = 0.2, gamma = 1, max_depth = 3, subsample = 1.0, colsample_bytree = 0.9, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_model.fit(X_temp, y_temp)
y_pred_xgb_no = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred_xgb_no))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [27]:
np.random.seed(2025)
xgb_model.fit(X_full_no, y)
y_pred_xgb_no = xgb_model.predict(X_test_full_no)

In [28]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_xgb_no})

with open('XGB_Full_No_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [29]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [30]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [31]:
np.random.seed(2025)
xgb_grid = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

Best Parameters: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.1, 'gamma': 0, 'max_depth': 3, 'n_estimators': 300, 'scale_pos_weight': 1, 'subsample': 0.8}


In [32]:
np.random.seed(2025)
xgb_model = XGBClassifier(n_estimators = 300, eta = 0.1, gamma = 0, max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_model.fit(X_temp, y_temp)
y_pred_manual = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [33]:
np.random.seed(2025)
xgb_model.fit(X_no_manual, y)
y_pred_xgb_no_manual = xgb_model.predict(X_test_no_manual)

In [34]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_xgb_no_manual})

with open('XGB_No_Manual_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [35]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [36]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [37]:
np.random.seed(2025)
xgb_grid = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

Best Parameters: {'booster': 'dart', 'colsample_bytree': 0.8, 'eta': 0.1, 'gamma': 1, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 5, 'subsample': 0.9}


In [38]:
np.random.seed(2025)
xgb_model = XGBClassifier(n_estimators = 100, eta = 0.1, gamma = 1, max_depth = 3, subsample = 0.9, colsample_bytree = 0.8, booster = 'dart', scale_pos_weight = 5, random_state = 2025)
xgb_model.fit(X_temp, y_temp)
y_pred_manual = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [39]:
np.random.seed(2025)
xgb_model.fit(X_manual, y)
y_pred_xgb_manual = xgb_model.predict(X_manual_test)

In [40]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_xgb_manual})

with open('XGB_Manual_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SMOTE FOR BEST XGB:

In [41]:
from imblearn.over_sampling import SMOTE

In [42]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

smote = SMOTE(random_state = 2025)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

In [45]:
np.random.seed(2025)
xgb_grid = GridSearchCV(XGBClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

Best Parameters: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.2, 'gamma': 0, 'max_depth': 3, 'n_estimators': 200, 'scale_pos_weight': 1, 'subsample': 0.8}


In [47]:
np.random.seed(2025)
xgb_model_smote = XGBClassifier(n_estimators = 200, eta = 0.2, gamma = 0, max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_model_smote.fit(X_resampled, y_resampled)
y_pred_smote = xgb_model_smote.predict(X_test)

print(classification_report(y_test, y_pred_smote))
y_pred_smote_xgb = xgb_model_smote.predict(X_test_full)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [48]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_smote_xgb})

with open('XGB_SMOTE_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_SMOTE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GHOST:


In [49]:
!pip install ghostml

In [50]:
import ghostml
from sklearn import metrics

def calc_metrics(labels_test, test_probs, threshold = 0.5):
  scores = [1 if x>=threshold else 0 for x in test_probs]
  auc = metrics.roc_auc_score(labels_test, test_probs)
  kappa = metrics.cohen_kappa_score(labels_test, scores)
  confusion = metrics.confusion_matrix(labels_test, scores, labels = list(set(labels_test)))

  print('Threshold: %.2f, kappa: %.3f, AUC test set: %.3f'%(threshold, kappa, auc))
  print(confusion)
  print(metrics.classification_report(labels_test, scores))

  return

thresholds = np.round(np.arange(0.01, 0.75,0.01),2)

In [51]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

xgb_ghost = XGBClassifier(n_estimators = 200, eta = 0.2, gamma = 0, max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, booster = 'gbtree', scale_pos_weight = 1, random_state = 2025)
xgb_ghost.fit(X_temp, y_temp)
xgb_test_probs = xgb_ghost.predict_proba(X_test)[:,1]
calc_metrics(y_test, xgb_test_probs, threshold = 0.5)

Threshold: 0.50, kappa: 1.000, AUC test set: 1.000
[[36  0]
 [ 0  9]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [52]:
np.random.seed(2025)
xgb_temp_probs = xgb_ghost.predict_proba(X_temp)[:,1]
xgb_threshold = ghostml.optimize_threshold_from_predictions(y_temp, xgb_temp_probs, thresholds, ThOpt_metrics = 'Kappa')

np.random.seed(2025)
calc_metrics(y_test, xgb_test_probs, threshold = xgb_threshold)
xgb_GHOST_test = [1 if x >= xgb_threshold else 0 for x in xgb_test_probs]

Threshold: 0.07, kappa: 0.872, AUC test set: 1.000
[[34  2]
 [ 0  9]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        36
           1       0.82      1.00      0.90         9

    accuracy                           0.96        45
   macro avg       0.91      0.97      0.94        45
weighted avg       0.96      0.96      0.96        45



In [53]:
np.random.seed(2025)
xgb_ghost.fit(X_full, y)
xgb_probs = xgb_ghost.predict_proba(X_test_full)[:,1]
xgb_pred_ghost = [1 if x >= xgb_threshold else 0 for x in xgb_probs]

In [55]:
from google.colab import files

pred_rf = pd.DataFrame({'ID': range(1, 92), 'Placement': xgb_pred_ghost})

with open('XGB_GHOST_Cov.csv', 'w') as file:
    pred_rf.to_csv(file, index = False, header = True)

files.download('XGB_GHOST_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>